In [6]:
import pandas as pd
import numpy as np 
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot
import datetime as dt
#for offline plotting
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 
df = pd.read_csv('C:/Users/sakthi/Documents/ML/Project/stock_data.csv',header=0)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2566 entries, 0 to 2565
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2566 non-null   object 
 1   Open    2566 non-null   float64
 2   High    2566 non-null   float64
 3   Low     2566 non-null   float64
 4   Close   2566 non-null   float64
 5   Volume  2566 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 120.4+ KB


In [8]:
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)
df['Date'] = df['Date'].map(dt.datetime.toordinal)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2566 entries, 0 to 2565
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2566 non-null   int64  
 1   Open    2566 non-null   float64
 2   High    2566 non-null   float64
 3   Low     2566 non-null   float64
 4   Close   2566 non-null   float64
 5   Volume  2566 non-null   int64  
dtypes: float64(4), int64(2)
memory usage: 120.4 KB


# Test-Train 

In [10]:
x = df.loc[:,df.columns!="Close"]

In [11]:
type(x)

pandas.core.frame.DataFrame

In [12]:
y=df["Close"]

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

### Training

In [67]:
from sklearn import tree

In [68]:
clftree = tree.DecisionTreeClassifier(max_depth = 3)

In [69]:
clftree.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=3)

In [70]:
y_train_pred = clftree.predict(x_train)
y_test_pred = clftree.predict(x_test)

## Perfomance Measures

In [71]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [72]:
confusion_matrix(y_train,y_train_pred)

array([[172,  14],
       [126,  92]], dtype=int64)

In [73]:
confusion_matrix(y_test,y_test_pred)

array([[39,  5],
       [41, 17]], dtype=int64)

In [74]:
accuracy_score(y_train,y_train_pred)

0.6534653465346535

In [75]:
accuracy_score(y_test,y_test_pred)

0.5490196078431373

# Bagging

In [76]:
from sklearn.ensemble import BaggingClassifier

In [77]:
clstree = tree.DecisionTreeClassifier

In [78]:
bag_clf = BaggingClassifier(estimator = clftree,n_estimators = 1000,
                           n_jobs=-1,random_state=42, bootstrap=True)

In [79]:
bag_clf.fit(x_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=3),
                  n_estimators=1000, n_jobs=-1, random_state=42)

In [80]:
confusion_matrix(y_test,bag_clf.predict(x_test))

array([[35,  9],
       [32, 26]], dtype=int64)

In [81]:
accuracy_score(y_test,bag_clf.predict(x_test))

0.5980392156862745

# Random Forest

In [19]:
from sklearn.model_selection import train_test_split

#For preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#For model evaluation
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [20]:
from sklearn.ensemble import RandomForestRegressor
clf_rf  =  RandomForestRegressor(n_estimators = 1000,
                           n_jobs=-1,random_state=42, bootstrap=True)
clf_rf.fit(x_train,y_train)

RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)

In [24]:
#Calculate scores for model evaluation
scores = f'''
{'Metric'.ljust(10)}{'Train'.center(20)}{'Test'.center(20)}
{'r2_score'.ljust(10)}{r2_score(y_train,clf_rf.predict(x_train))}\t{r2_score(y_test,clf_rf.predict(x_test))}
{'MSE'.ljust(10)}{mse(y_train,clf_rf.predict(x_train))}\t{mse(y_test,clf_rf.predict(x_test))}
'''
print(scores)


Metric           Train                Test        
r2_score  0.9999149155446875	0.9993725075019826
MSE       0.8832202532943786	6.71228028345921



## Grid Search

In [85]:
from sklearn.model_selection import GridSearchCV

In [86]:
rf_clf = RandomForestClassifier(n_estimators=250,random_state=42)


In [87]:
params_grid = {"max_features":[4,5,6,7,8,9,10],"min_samples_split":[2,3,10]}

In [88]:
grid_search = GridSearchCV(rf_clf,params_grid,n_jobs=-1,cv=5,scoring = 'accuracy')

In [89]:
grid_search.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=250,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_features': [4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 3, 10]},
             scoring='accuracy')

In [90]:
grid_search.best_params_

{'max_features': 6, 'min_samples_split': 3}

In [91]:
cvrf_clf=grid_search.best_estimator_

In [92]:
confusion_matrix(y_test,cvrf_clf.predict(x_test))

array([[25, 19],
       [20, 38]], dtype=int64)

In [93]:
accuracy_score(y_test,cvrf_clf.predict(x_test))

0.6176470588235294